In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)

# from    tensorflow.keras import layers, optimizers, datasets

2.0.0-rc1


In [29]:
(data_train, label_train), (data_valid, label_valid) = keras.datasets.fashion_mnist.load_data()
print(data_train.shape,label_train.shape)
print(set(label_train))
print(data_valid.shape,data_valid.shape)
# dataset = tf.data.Dataset.from_tensor_slices((data, label))
# dataset = dataset.map(prepare_mnist_features_and_labels)
# dataset = dataset.take(20000).shuffle(20000).batch(100)

(60000, 28, 28) (60000,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
(10000, 28, 28) (10000, 28, 28)


In [44]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y
def make_datase(data,label):
    dataset=tf.data.Dataset.from_tensor_slices((data, label))
    dataset = dataset.map(prepare_mnist_features_and_labels)
    dataset = dataset.shuffle(10000).batch(100)
    return dataset


dataset_train = make_datase(data_train, label_train)
dataset_valid = make_datase(data_valid, label_valid)


model method 1

In [32]:
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))

def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

def batch_train(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    # update to weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

def train(model, optimizer, dataset):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(dataset):
        loss, accuracy = batch_train(model, optimizer, x, y)
#         if step%5000==0:
#             #print("step={:0>6d}\tloss={:.6f}\tacc={:.4f}".format(step,loss.numpy,accuracy.numpy))
#             print('step', step, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
    return loss, accuracy

In [33]:
model = keras.Sequential([keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
                          keras.layers.Dense(200, activation='relu'),
                          keras.layers.Dense(200, activation='relu'),
                          keras.layers.Dense(10)])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2010      
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
_________________________________________________________________


In [34]:
optimizer = keras.optimizers.Adam()
for epoch in range(5):
    loss, accuracy = train(model, optimizer, dataset_train)
    #print("\nepoch={:0>6d}\tloss={:.6f}\tacc={:.4f}".format(epoch,loss.numpy,accuracy.numpy))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.61457676 ; accuracy 0.77
epoch 1 : loss 0.3720266 ; accuracy 0.86
epoch 2 : loss 0.50202155 ; accuracy 0.84
epoch 3 : loss 0.4606101 ; accuracy 0.81
epoch 4 : loss 0.39552906 ; accuracy 0.86


model mothed 2

In [37]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = keras.layers.Dense(200, activation=tf.nn.relu)
        self.layer2 = keras.layers.Dense(200, activation=tf.nn.relu)
        self.layer3 = keras.layers.Dense(10)

    def call(self, x, training=False):
        x = tf.reshape(x, [-1, 28*28])
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [41]:
model = MyModel()
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model.summary()

In [45]:
model.fit(dataset_train.repeat(), epochs=5, steps_per_epoch=500, verbose=1,
              validation_data=dataset_valid.repeat(),
              validation_steps=2)

Train for 500 steps, validate for 2 steps
Epoch 1/5
500/500 [==============================] - 3s 7ms/step - loss: 797121.0500 - accuracy: 0.0000e+00 - val_loss: 2528534.3750 - val_accuracy: 0.0000e+00
Epoch 2/5
500/500 [==============================] - 3s 5ms/step - loss: 5994458.6615 - accuracy: 0.0000e+00 - val_loss: 10559857.5000 - val_accuracy: 0.0000e+00
Epoch 3/5
500/500 [==============================] - 3s 5ms/step - loss: 14362181.3790 - accuracy: 0.0000e+00 - val_loss: 17303422.5000 - val_accuracy: 0.0000e+00
Epoch 4/5
500/500 [==============================] - 3s 5ms/step - loss: 23401485.7280 - accuracy: 0.0000e+00 - val_loss: 26213081.0000 - val_accuracy: 0.0000e+00
Epoch 5/5
500/500 [==============================] - 3s 5ms/step - loss: 33408497.6440 - accuracy: 0.0000e+00 - val_loss: 46919874.0000 - val_accuracy: 0.0000e+00


In [ ]:
model mothed 3

In [47]:
model = keras.Sequential([
        keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
        keras.layers.Dense(200, activation='relu'),
        keras.layers.Dense(200, activation='relu'),
        keras.layers.Dense(200, activation='relu'),
        keras.layers.Dense(10)])
# no need to use compile if you have no loss/optimizer/metrics involved here.
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 200)               157000    
_________________________________________________________________
dense_27 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_28 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_29 (Dense)             (None, 10)                2010      
Total params: 239,410
Trainable params: 239,410
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(dataset_train.repeat(), epochs=5, steps_per_epoch=500, verbose=1,
              validation_data=dataset_valid.repeat(),
              validation_steps=2)

Train for 500 steps, validate for 2 steps
Epoch 1/5
500/500 [==============================] - 4s 7ms/step - loss: 93531316.0967 - accuracy: 0.9174 - val_loss: 383383056.0000 - val_accuracy: 1.0000
Epoch 2/5
500/500 [==============================] - 3s 6ms/step - loss: 1594857473.0240 - accuracy: 1.0000 - val_loss: 3387993472.0000 - val_accuracy: 1.0000
Epoch 3/5
500/500 [==============================] - 3s 6ms/step - loss: 6255194450.4320 - accuracy: 0.9880 - val_loss: 10364723200.0000 - val_accuracy: 1.0000
Epoch 4/5
500/500 [==============================] - 3s 6ms/step - loss: 14660614294.5280 - accuracy: 0.9796 - val_loss: 19018120192.0000 - val_accuracy: 1.0000
Epoch 5/5
500/500 [==============================] - 3s 6ms/step - loss: 26097454473.2160 - accuracy: 0.9723 - val_loss: 36084943872.0000 - val_accuracy: 1.0000


model method 4

In [50]:
class MyLayer(keras.layers.Layer):
    def __init__(self, units):
        """
        :param units: [input_dim, h1_dim,...,hn_dim, output_dim]
        """
        super(MyLayer, self).__init__()
        for i in range(1, len(units)):
            # w: [input_dim, output_dim]
            self.add_variable(name='kernel%d'%i, shape=[units[i-1], units[i]])
            # b: [output_dim]
            self.add_variable(name='bias%d'%i,shape=[units[i]])

    def call(self, x):
        """
        :param x: [b, input_dim]
        :return:
        """
        num = len(self.trainable_variables)
        x = tf.reshape(x, [-1, 28*28])
        for i in range(0, num, 2):
            x = tf.matmul(x, self.trainable_variables[i]) + self.trainable_variables[i+1]

        return x

In [52]:
model = MyLayer([28*28, 200, 200, 10])
for p in model.trainable_variables:
    print(p.name, p.shape)
    
optimizer = keras.optimizers.Adam()
for epoch in range(5):
    loss, accuracy = train(model, optimizer, dataset_train)
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

kernel1:0 (784, 200)
bias1:0 (200,)
kernel2:0 (200, 200)
bias2:0 (200,)
kernel3:0 (200, 10)
bias3:0 (10,)
epoch 0 : loss 0.3746219 ; accuracy 0.83
epoch 1 : loss 0.47332108 ; accuracy 0.82
epoch 2 : loss 0.42367095 ; accuracy 0.83
epoch 3 : loss 0.423033 ; accuracy 0.84
epoch 4 : loss 0.38566503 ; accuracy 0.88
